In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
- Load the csv exported in Part I to a DataFrame

In [2]:
load_csv = "cities.csv"
city_df = pd.read_csv(load_csv)
city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [3]:
city_df.count()

City_ID       548
City          548
Cloudiness    548
Country       544
Date          548
Humidity      548
Lat           548
Lng           548
Max Temp      548
Wind Speed    548
dtype: int64

### Humidity Heatmap
- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight.
- Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
city_df = city_df.dropna()
humidity = city_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = city_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure(map_type='SATELLITE')

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(city_df["Humidity"]),
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

- Narrow down the cities to fit weather conditions.
- Drop any rows will null values.

In [5]:
narrowed_city_df = city_df.copy()
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 70]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 80]  
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= 10]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] == 0]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df = narrowed_city_df.reset_index()
del narrowed_city_df['index']
narrowed_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
5,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
6,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [6]:
narrowed_city_df.count()

City_ID       7
City          7
Cloudiness    7
Country       7
Date          7
Humidity      7
Lat           7
Lng           7
Max Temp      7
Wind Speed    7
dtype: int64

### Hotel Map
- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
- Plot markers on top of the heatmap.

In [7]:
hotel_df = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")
narrowed_city_df["Hotel Name"] = hotel_df
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Capitólio
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,CIDREIRA
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Morondava
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Jagalbet
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Nador


In [8]:
hotel_df = narrowed_city_df
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Capitólio
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,CIDREIRA
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Morondava
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Jagalbet
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Nador


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
# Display figure

fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))